<a href="https://colab.research.google.com/github/UbaidullahTanoli/LLM-CNN/blob/main/LLM%2BCNN%2BMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext cuml.accel

[2025-03-30 08:53:50.307] [CUML] [info] cuML: Installed accelerator for sklearn.
[2025-03-30 08:54:01.430] [CUML] [info] cuML: Installed accelerator for umap.
[2025-03-30 08:54:01.443] [CUML] [info] cuML: Installed accelerator for hdbscan.
[2025-03-30 08:54:01.443] [CUML] [info] cuML: Successfully initialized accelerator.


In [ ]:
from google.colab import files
files.upload()

# 10:15am, compute = 70, system ram = 3.3, gpu ram = 0.4

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university
!unzip chest-xrays-indiana-university.zip -d /content/dataset/A

In [ ]:
#pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [10]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights
import matplotlib.pyplot as plt
from collections import defaultdict

In [11]:
class CombinedDataset(Dataset):
    def __init__(self, reports_csv, proj_csv, image_folder, max_length=256,
                 tokenizer_name="dmis-lab/biobert-base-cased-v1.1", transform=None,
                 is_training=True):
        """
        Args:
            reports_csv (str): Path to the reports CSV
            proj_csv (str): Path to the projections CSV
            image_folder (str): Folder containing the image files
            max_length (int): Maximum token length for the report text
            tokenizer_name (str): Name of the pretrained tokenizer
            transform (callable, optional): Image transformation
            is_training (bool): Whether this dataset is for training (for augmentation decisions)
        """
        # Load CSVs and merge on 'uid'
        self.reports_df = pd.read_csv(reports_csv)
        self.proj_df = pd.read_csv(proj_csv)
        self.data = pd.merge(self.reports_df, self.proj_df, on='uid')
        self.image_folder = image_folder
        self.is_training = is_training

        # Remove rows where both "findings" and "impression" are empty
        self.data = self.data[~(
            (self.data['findings'].isna() | (self.data['findings'].str.strip() == "")) &
            (self.data['impression'].isna() | (self.data['impression'].str.strip() == ""))
        )]

        # Initialize tokenizer for report text
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

        # Define transforms based on training/evaluation
        if transform is None:
            if is_training:
                # Use moderate augmentation for training
                self.transform = transforms.Compose([
                    transforms.Resize((240, 240)),  # Slightly larger for random crop

                    transforms.RandomCrop((224, 224)),

                    transforms.RandomRotation(degrees=7),
                    transforms.RandomAffine(degrees=7, translate=(0.07, 0.07), scale=(0.95, 1.05)),
                    transforms.ColorJitter(brightness=0.15, contrast=0.15),

                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                ])
            else:
                # No augmentation for validation/testing
                self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                ])
        else:
            self.transform = transform

        # Prepare labels from "MeSH"
        self.labels = []
        for _, row in self.data.iterrows():
            mesh_val = str(row['MeSH']).strip().lower()
            label = 0 if mesh_val == 'normal' else 1
            self.labels.append(label)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Get image from image folder using filename
        img_path = os.path.join(self.image_folder, row['filename'])
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = self.transform(image)

        # Concatenate text from "findings" and "impression"
        report_text = ""
        if 'findings' in row and not pd.isna(row['findings']):
            report_text += "Findings: " + str(row['findings']) + " "
        if 'impression' in row and not pd.isna(row['impression']):
            report_text += "Impression: " + str(row['impression'])
        if not report_text.strip():
            report_text = "No report text available."

        # Tokenize the report text
        tokenized = self.tokenizer(
            report_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        input_ids = tokenized['input_ids'].squeeze(0)
        attention_mask = tokenized['attention_mask'].squeeze(0)

        # Get label
        mesh_val = str(row['MeSH']).strip().lower()
        label = 0 if mesh_val == 'normal' else 1

        return {
            'image': image,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long),
            'uid': row['uid'] if 'uid' in row else -1
        }

    def get_class_distribution(self):
        """Returns a dictionary with the class distribution"""
        return dict(zip(*np.unique(self.labels, return_counts=True)))

In [12]:
class LLMCNNMLPClassifier(nn.Module):
    def __init__(self, num_classes=2, freeze_cnn=True, freeze_llm=True,
                 cnn_model_name="efficientnet_b1", llm_model_name="dmis-lab/biobert-base-cased-v1.1",
                 dropout_rate=0.5):
        super(LLMCNNMLPClassifier, self).__init__()

        # Load CNN backbone (EfficientNet-B1)
        weights = EfficientNet_B1_Weights.IMAGENET1K_V1
        self.cnn = efficientnet_b1(weights=weights)

        # Freeze CNN initially
        if freeze_cnn:
            for param in self.cnn.parameters():
                param.requires_grad = False

        # Store the freezing state for later unfreezing specific layers
        self.cnn_frozen = freeze_cnn

        # Pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Load LLM backbone (BioBERT)
        self.llm = AutoModel.from_pretrained(llm_model_name)
        if freeze_llm:
            for param in self.llm.parameters():
                param.requires_grad = False

        # Improved MLP head with additional layer and higher dropout
        self.mlp = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )

        # Initialize weights properly
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize the weights of the MLP head using He initialization"""
        for m in self.mlp.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def unfreeze_cnn_last_layer(self):
        """Unfreeze only the last layer of the CNN"""
        # Freeze all layers
        for param in self.cnn.parameters():
            param.requires_grad = False

        # Unfreeze only the final blocks
        for param in self.cnn.features[-1].parameters():
            param.requires_grad = True
        for param in self.cnn.classifier.parameters():
            param.requires_grad = True

        self.cnn_frozen = False

    def forward(self, input_ids, attention_mask, image):
        # CNN branch: extract features from image
        cnn_features = self.cnn.features(image)  # shape: (B, 1280, H, W)
        pooled_cnn = self.avgpool(cnn_features)  # shape: (B, 1280, 1, 1)
        cnn_vector = pooled_cnn.flatten(1)       # shape: (B, 1280)

        # LLM branch: get outputs from BioBERT
        outputs = self.llm(input_ids=input_ids, attention_mask=attention_mask)

        # Get averaged hidden states for a better representation
        llm_vector = outputs.last_hidden_state.mean(dim=1)  # shape: (B, 768)

        # Concatenate both feature vectors
        combined = torch.cat((cnn_vector, llm_vector), dim=1)  # shape: (B, 2048)

        # Pass through MLP head
        logits = self.mlp(combined)
        return logits

In [13]:
def compute_metrics(labels, preds, probs):
    """Compute various classification metrics"""
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)

    # Compute confusion matrix
    cm = confusion_matrix(labels, preds)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Compute AUC metrics with error handling
    try:
        roc_auc = roc_auc_score(labels, probs)
        pr_auc = average_precision_score(labels, probs)
    except ValueError:
        roc_auc = 0.0
        pr_auc = 0.0

    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'specificity': specificity,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    }

In [14]:
def train_epoch(model, dataloader, criterion, optimizer, device, scaler=None, epoch=None):
    """Train the model for one epoch"""
    model.train()
    running_loss = 0.0
    preds_all, labels_all, probs_all = [], [], []

    progress_bar = tqdm(dataloader, desc=f"Training Epoch {epoch}", dynamic_ncols=True)
    for batch in progress_bar:
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Use mixed precision
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(input_ids, attention_mask, images)
            loss = criterion(outputs, labels)

        # Scale gradients and optimize
        scaler.scale(loss).backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        # Update metrics
        running_loss += loss.item() * images.size(0)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        _, preds = torch.max(outputs, dim=1)

        preds_all.extend(preds.cpu().numpy())
        labels_all.extend(labels.cpu().numpy())
        probs_all.extend(probs.detach().cpu().numpy())

        # Update progress bar
        progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss

    return metrics

In [15]:
def eval_epoch(model, dataloader, criterion, device, epoch=None):
    """Evaluate the model for one epoch"""
    model.eval()
    running_loss = 0.0
    preds_all, labels_all, probs_all = [], [], []

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc=f"Evaluating Epoch {epoch}", dynamic_ncols=True)
        for batch in progress_bar:
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(input_ids, attention_mask, images)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            _, preds = torch.max(outputs, dim=1)

            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())
            probs_all.extend(probs.detach().cpu().numpy())

            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss

    return metrics

In [16]:
def train_and_evaluate(reports_csv, proj_csv, image_folder,
                      num_epochs=40, batch_size=8, learning_rate=1e-4,
                      test_size=0.2, random_state=42, unfreeze_cnn_at_epoch=15,
                      dropout_rate=0.5, weight_decay=0.01, early_stopping_patience=7
                      ):
    """Train and evaluate the LLM+CNN+MLP model"""
    # Set up device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Define transforms based on training/evaluation
    train_transform = transforms.Compose([
        transforms.Resize((240, 240)),  # Slightly larger for random crop
        transforms.RandomCrop((224, 224)),
        transforms.RandomRotation(degrees=7),
        transforms.RandomAffine(degrees=7, translate=(0.07, 0.07), scale=(0.95, 1.05)),
        transforms.ColorJitter(brightness=0.15, contrast=0.15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
    ])

    full_dataset = CombinedDataset(reports_csv, proj_csv, image_folder, transform=None, is_training=False)
    class_distribution = full_dataset.get_class_distribution()
    print(f"Class distribution: {class_distribution}")

    # Create stratified train/val split
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)

    train_indices, val_indices = next(split.split(np.zeros(len(full_dataset)), full_dataset.labels))

    # Create train and validation datasets with their appropriate transforms
    train_dataset = CombinedDataset(reports_csv, proj_csv, image_folder, transform=train_transform, is_training=True)
    val_dataset = CombinedDataset(reports_csv, proj_csv, image_folder, transform=val_transform, is_training=False)

    # Create subsets using the indices
    train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(val_dataset, val_indices)

    # Calculate class weights for balanced loss
    train_labels = [full_dataset.labels[i] for i in train_indices]
    label_counts = {}
    for label in train_labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

    num_samples = len(train_labels)
    num_classes = len(label_counts)
    class_weights = torch.FloatTensor([num_samples / (num_classes * label_counts[label]) for label in sorted(label_counts.keys())]).to(device)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, pin_memory=True, drop_last=False
    )

    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=8, pin_memory=True, drop_last=False
    )

    # Initialize model
    model = LLMCNNMLPClassifier(
        num_classes=len(class_distribution),
        freeze_cnn=True,
        freeze_llm=True,
        dropout_rate=dropout_rate
    ).to(device)

    # Loss function with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # Two parameter groups for different learning rates
    optimizer = optim.AdamW([
        {'params': model.mlp.parameters(), 'lr': learning_rate}
    ], weight_decay=weight_decay)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )

    # Initialize gradient scaler for mixed precision
    scaler = GradScaler()

    # Track best model for early stopping
    best_combined_metric = 0.0
    best_epoch = 0
    early_stopping_counter = 0

    # For plotting
    train_metrics_history = defaultdict(list)
    val_metrics_history = defaultdict(list)

    # Training loop
    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}")

        # Unfreeze CNN layers at specified epoch
        if epoch == unfreeze_cnn_at_epoch and model.cnn_frozen:
            print("Unfreezing last layer of CNN...")
            model.unfreeze_cnn_last_layer()

            # Add CNN parameters to optimizer with a lower learning rate
            optimizer.add_param_group({
                'params': [p for p in model.cnn.parameters() if p.requires_grad],
                'lr': learning_rate / 10
            })

        # Train for one epoch
        train_metrics = train_epoch(model, train_loader, criterion, optimizer, device, scaler, epoch)

        # Evaluate
        val_metrics = eval_epoch(model, val_loader, criterion, device, epoch)

        # Store metrics history
        for metric in train_metrics:
            train_metrics_history[metric].append(train_metrics[metric])
            val_metrics_history[metric].append(val_metrics[metric])

        # Print all the metrics
        print(f"Train - Loss: {train_metrics['loss']:.4f} | Acc: {train_metrics['accuracy']:.4f} | "
              f"F1: {train_metrics['f1']:.4f} | Prec: {train_metrics['precision']:.4f} | "
              f"Rec: {train_metrics['recall']:.4f} | Spec: {train_metrics['specificity']:.4f} | "
              f"ROC-AUC: {train_metrics['roc_auc']:.4f} | PR-AUC: {train_metrics['pr_auc']:.4f}")
        print(f"Val   - Loss: {val_metrics['loss']:.4f} | Acc: {val_metrics['accuracy']:.4f} | "
              f"F1: {val_metrics['f1']:.4f} | Prec: {val_metrics['precision']:.4f} | "
              f"Rec: {val_metrics['recall']:.4f} | Spec: {val_metrics['specificity']:.4f} | "
              f"ROC-AUC: {val_metrics['roc_auc']:.4f} | PR-AUC: {val_metrics['pr_auc']:.4f}")

        # Calculate a combined metric
        combined_metric = 0.5 * val_metrics['f1'] + 0.5 * val_metrics['roc_auc']

        # Track best combined metric
        if combined_metric > best_combined_metric:
            best_combined_metric = combined_metric
            best_epoch = epoch
            early_stopping_counter = 0
            print(f"New best model based on combined metric: {combined_metric:.4f} (F1: {val_metrics['f1']:.4f}, ROC-AUC: {val_metrics['roc_auc']:.4f})")

            # Save the best model
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            early_stopping_counter += 1
            print(f"No improvement for {early_stopping_counter}/{early_stopping_patience} epochs")

            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered after {epoch} epochs")
                break

    # Final evaluation
    final_val_metrics = eval_epoch(model, val_loader, criterion, device)

    print("\nTraining Complete!")
    print(f"Best model was from epoch {best_epoch} with combined metric: {best_combined_metric:.4f}")
    print("\nFinal Validation Metrics:")
    for metric, value in final_val_metrics.items():
        print(f"  {metric}: {value:.4f}")

    return model, final_val_metrics, (train_metrics_history, val_metrics_history)

In [17]:
def main():
    """Main function to run the training and evaluation"""
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Define data paths
    reports_csv = "/content/dataset/indiana_reports.csv"
    proj_csv = "/content/dataset/indiana_projections.csv"
    image_folder = "/content/dataset/images/images_normalized"

    # Train and evaluate model with optimized parameters
    model, final_metrics, metrics_history = train_and_evaluate(
        reports_csv=reports_csv,
        proj_csv=proj_csv,
        image_folder=image_folder,
        num_epochs=40,
        batch_size=8,
        learning_rate=3e-4,  # Slightly lower learning rate
        test_size=0.2,
        random_state=42,
        unfreeze_cnn_at_epoch=15,
        dropout_rate=0.6,     # Higher dropout for better regularization
        weight_decay=0.01,    # L2 regularization
        early_stopping_patience=7,
    )

    print("Training and evaluation complete!")

In [18]:
if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Class distribution: {np.int64(0): np.int64(2655), np.int64(1): np.int64(4771)}


model.safetensors:  51%|#####     | 220M/436M [00:00<?, ?B/s]


Epoch 1/40


...

Training Epoch 21:   0%|          | 0/743 [00:00<?, ?it/s]

Evaluating Epoch 21:   0%|          | 0/186 [00:00<?, ?it/s]

Train - Loss: 1.0086 | Acc: 0.8582 | F1: 0.8893 | Prec: 0.8923 | Rec: 0.8863 | Spec: 0.8079 | ROC-AUC: 0.9166 | PR-AUC: 0.9406
Val   - Loss: 0.3714 | Acc: 0.8829 | F1: 0.9015 | Prec: 0.9815 | Rec: 0.8335 | Spec: 0.9718 | ROC-AUC: 0.9762 | PR-AUC: 0.9851
No improvement for 7/7 epochs
Early stopping triggered after 21 epochs


Evaluating Epoch None:   0%|          | 0/186 [00:00<?, ?it/s]


Training Complete!
Best model was from epoch 14 with combined metric: 0.9537

Final Validation Metrics:
  accuracy: 0.8829
  precision: 0.9815
  recall: 0.8335
  f1: 0.9015
  specificity: 0.9718
  roc_auc: 0.9762
  pr_auc: 0.9851
  loss: 0.3714
Training and evaluation complete!
